## Khoa Học Tự Nhiên
## Lab 4 - ML - Naive Bayes
### Nguyễn Quốc Bảo - 18110053







**lab 04: Naive Bayes**

Ta  có $X_i=(x_i^1,x_i^2,...,x_i^n)$ là một sample có n features, với $c$ labels ($C_1,C_2,...C_c$). Ta muốn chọn được label cho sample $X_i$ thì dựa vào:
$$C=\mathop{argmax}_{C_j, j=\overline{1,c}}P(C_j|X_i)$$
Giả sử các thành phần của biến ngẫu nhiên X độc lập với nhau, ta được: 
$$C=\mathop{argmax}_{C_j, j=\overline{1,c}}P(C_j)\prod_{i=1}^n P(X_i|C_j)$$

Tóm lại: với mỗi label $C_j$ ta tính tích $P(C_j)\prod_{i=1}^n P(X_i|C_j)$ và so sánh chúng với nhau từ đó chọn ra được label mang giá trị lớn nhất

Một số phân phối:
1. Categorical

2. Gaussian


#### Bài tập
1. Dùng các features: Sex, Pclass, và Embarked để xây dựng thuật toán Naive Bayes Categorical. (4đ)
2. Dùng các features: SibSp,	Parch, và Fare để xây dựng thuật toán Naive Bayes Gaussian (Khỏi normalize). (4đ)
3. Từ bộ dữ liệu liên minh hãy xây dựng thuật toán Naive Bayes bằng cách dùng tất cả features (mix cả Multinomial và Gaussian. Lưu ý: Khỏi chia tập train, test. Dự đoán thẳng trên tập train) (2đ)

In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv('https://raw.githubusercontent.com/huynhthanh98/ML/master/lab-04/train.csv')
print(data.shape)
data.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Đây là data thống kê các hành khách trên con tàu Titanic, trong đó:
PassengerId là Id hành khách

Survived là sống sót hay không (giá trị dự đoán). 1 là có, 0 là không

Pclass: Loại vé [1,2,3]

Name tên hành khách

Age tuổi hành khách

Parch: Số anh cha mẹ/con trên tàu

SibSp: Số anh chị em/ vợ chông trên tàu

ticket: mã  vé

fare: Phí hành khách

cabin: số khoang

Embarked: Cảng lên tàu

Chúng ta sẽ dự đoán với tất cả features trên thì ai có khả năng sống sót được.


In [3]:
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [4]:
data=data.drop(["Cabin","Age"],axis=1)
data=data.dropna()
data.shape

(889, 10)

In [5]:
#Phân tích thử các giá trị có trong mỗi feature
for i in data.drop(labels=["Name","Ticket","PassengerId"],axis=1).columns:
    print(i)
    print(data[i].unique())

Survived
[0 1]
Pclass
[3 1 2]
Sex
['male' 'female']
SibSp
[1 0 3 4 2 5 8]
Parch
[0 1 2 5 3 4 6]
Fare
[  7.25    71.2833   7.925   53.1      8.05     8.4583  51.8625  21.075
  11.1333  30.0708  16.7     26.55    31.275    7.8542  16.      29.125
  13.      18.       7.225   26.       8.0292  35.5     31.3875 263.
   7.8792   7.8958  27.7208 146.5208   7.75    10.5     82.1708  52.
   7.2292  11.2417   9.475   21.      41.5792  15.5     21.6792  17.8
  39.6875   7.8     76.7292  61.9792  27.75    46.9     83.475   27.9
  15.2458   8.1583   8.6625  73.5     14.4542  56.4958   7.65    29.
  12.475    9.       9.5      7.7875  47.1     15.85    34.375   61.175
  20.575   34.6542  63.3583  23.      77.2875   8.6542   7.775   24.15
   9.825   14.4583 247.5208   7.1417  22.3583   6.975    7.05    14.5
  15.0458  26.2833   9.2167  79.2      6.75    11.5     36.75     7.7958
  12.525   66.6      7.3125  61.3792   7.7333  69.55    16.1     15.75
  20.525   55.      25.925   33.5     30.6958  25.4

## Categorical
* P(Survived)*P(X|Survived)


* P(notSurvived)*P(X|notSurvived)


#### 1. Dùng các features: Sex, Pclass, và Embarked để xây dựng thuật toán Naive Bayes Categorical. (4đ)

In [6]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,0,0,373450,8.0500,S


In [7]:
data.shape

(889, 10)

In [8]:
df = data

In [9]:
X = df.drop(['PassengerId',"Ticket", 'Name','Survived'],axis = 1)
y = df['Survived']

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X,y ,test_size=0.2
                                                    , random_state=42)

# y_train = y_train.values
y_test = y_test.values

In [11]:
print(X_train.shape)
print(X_test.shape)

(711, 6)
(178, 6)


In [12]:
X_train.columns

Index(['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')

In [13]:
###########################
# Train Naive Bayes Model
###########################
def CategoricalNB_model(X, y, laplace=True):
    # Calculate Prior Probability
    y_unique = y.unique()
    prior_probability = {}
    for i in range(0,len(y_unique)):
        prior_probability[y_unique[i]] = sum(y == y_unique[i])/len(y)
    
    # Calculate Conditional Probability
    conditional_probability = {}
    for col in X.columns:
        x_unique = list(set(X[col]))
        x_conditional_probability = np.zeros((len(y.unique()),len(set(X[col]))))
        for j in range(0,len(y_unique)):
            for k in range(0,len(x_unique)):
                if (laplace==True):
                    x_conditional_probability[j,k] = (X.loc[(X[col]==x_unique[k])&(y==y_unique[j]),].shape[0] + 1)/(sum(y==y_unique[j]) + len(x_unique))
                else:
                    x_conditional_probability[j,k] = X.loc[(X[col]==x_unique[k])&(y==y_unique[j]),].shape[0]/sum(y==y_unique[j])

        x_conditional_probability = pd.DataFrame(x_conditional_probability,columns=x_unique,index=y_unique)   
        conditional_probability[col] = x_conditional_probability       
    
    return prior_probability,conditional_probability


prior_probability,conditional_probability = CategoricalNB_model(X_train[['Sex', 'Pclass', 'Embarked']], y_train)
# print(conditional_probability)

In [14]:
for col in X_train[['Sex', 'Pclass', 'Embarked']].columns:
    print(col, conditional_probability[col].shape)

Sex (2, 2)
Pclass (2, 3)
Embarked (2, 3)


In [15]:
print(conditional_probability['Sex'])

       male    female
1  0.344322  0.655678
0  0.859729  0.140271


In [16]:
prior_probability

{1: 0.38115330520393814, 0: 0.6188466947960619}

In [17]:
list(prior_probability.values())[1]

0.6188466947960619

In [18]:
####################
# Prediction
####################
def prediction_CategoricalNB(X_test, conditional_probability, prior_probability):
    y_pred = []
    for i in range(len(X_test)):
        p0 = list(prior_probability.values())[1]  # P(notSurvived)
        p1 = list(prior_probability.values())[0]  # P(Survived)
        for col in X_test.columns:        
            p0 = p0 *conditional_probability[col][X_test[col].values[i]][0]
            p1 = p1 *conditional_probability[col][X_test[col].values[i]][1]
        if p0>p1:
            y_pred.append(0)
        else:
            y_pred.append(1)
    
    return np.array(y_pred)
    
y_pred = prediction_CategoricalNB(X_test[['Sex', 'Pclass', 'Embarked']], conditional_probability, prior_probability)

In [19]:
y_test

array([0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 1], dtype=int64)

In [20]:
y_pred

array([0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 1])

In [21]:
np.mean(y_pred == y_test)

0.7808988764044944

In [22]:
from sklearn.preprocessing import LabelEncoder
def Encoder(data):
    '''
    Encode target data
    Parameters:
        data: dataframe nead encode
    Returns:
        data: datafram encoded
    '''
    labelencoder = LabelEncoder()
    data_colums = data.dtypes.pipe(lambda X: X[X=='object']).index
    for column in data_colums:
        labelencoder.fit(data[column])
        data[column] = labelencoder.transform(data[column])
    return data

In [23]:
data_train = Encoder(X_train)
data_test = Encoder(X_test)

<ipython-input-22-e4fe50279b9b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column] = labelencoder.transform(data[column])


In [24]:
data_train.dtypes

Pclass        int64
Sex           int32
SibSp         int64
Parch         int64
Fare        float64
Embarked      int32
dtype: object

In [25]:
print(data_train.shape)
print(data_test.shape)

(711, 6)
(178, 6)


In [26]:
## Dùng sklearn
from sklearn.naive_bayes import CategoricalNB
MNB = CategoricalNB().fit(data_train[['Sex','Pclass','Embarked']], y_train)
print(np.mean(MNB.predict(data_test[['Sex','Pclass','Embarked']]) == y_test))
print(MNB.score(data_test[['Sex','Pclass','Embarked']],y_test))

0.7808988764044944
0.7808988764044944


## Gaussian
$$p(x_i|c)=P(x_i|\mu_{ci},\sigma_{ci}^2)=\frac{1}{\sqrt{2\pi\sigma_{ci}^2}}exp(-\frac{(x_i-\mu_{ci})^2}{2\sigma_{ci}^2})$$

#### 2. Dùng các features: SibSp,	Parch, và Fare để xây dựng thuật toán Naive Bayes Gaussian (Khỏi normalize). (4đ)

In [27]:
def gaussian(x, mean, std):
    return (1/(np.sqrt(2*np.pi)*std))*np.exp(-(np.square(x-mean))/(2*np.square(std)))

def GaussianNB_model(X,y):
    # Calculate Prior Probability
    y_unique = y.unique()
    prior_probability = {}
    for i in range(0,len(y_unique)):
        prior_probability[y_unique[i]] = sum(y == y_unique[i])/len(y)
    
    # Calculate Conditional Probability
    conditional_probability = {}  
    for col in X.columns:
        x_conditional_probability = np.zeros((len(y.unique()),2))
        Mu = {}
        Std = {}
        
        for j in range(0,len(y_unique)):    
            Mu_ci = np.mean(X.loc[y==y_unique[j]][[col]])
            Mu[j] = Mu_ci
            Std_ci = np.std(X.loc[y==y_unique[j]][[col]], ddof = 1)
            Std[j] = Std_ci
            x_conditional_probability[j] =  Mu[j], Std[j]
            
        x_conditional_probability = pd.DataFrame(x_conditional_probability,columns=['Mean','Std'],index=y_unique)   
        conditional_probability[col] = x_conditional_probability
    
    return prior_probability,conditional_probability


prior_probability,conditional_probability = GaussianNB_model(X_train[['SibSp', 'Parch','Fare']], y_train)

In [28]:
conditional_probability['SibSp']

,Mean,Std
1,0.464945,0.692331
0,0.609091,1.380604


In [29]:
prior_probability

{1: 0.38115330520393814, 0: 0.6188466947960619}

In [30]:
####################
# Prediction
####################
def prediction_GaussianNB(X_test, conditional_probability, prior_probability):
    y_pred = []
    for i in range(len(X_test)):
        p0 = list(prior_probability.values())[1] # P(notSurvived)
        p1 = list(prior_probability.values())[0] # P(Survived)
        for col in X_test.columns:
            p0 = p0*gaussian(X_test[col].values[i], conditional_probability[col]['Mean'][0],conditional_probability[col]['Std'][0])
            p1 = p1*gaussian(X_test[col].values[i], conditional_probability[col]['Mean'][1],conditional_probability[col]['Std'][1])
        if p0 > p1:
            y_pred.append(0)
        else:
            y_pred.append(1)
    return np.array(y_pred)

pred = prediction_GaussianNB(X_test[['SibSp', 'Parch','Fare']], conditional_probability, prior_probability)

In [31]:
pred

array([0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

In [32]:
y_test

array([0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 1], dtype=int64)

In [33]:
np.mean(pred == y_test)

0.6910112359550562

In [34]:
## Dùng sklearn
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB().fit(X_train[['SibSp', 'Parch','Fare']], y_train)
print(np.mean(y_test == nb.predict(X_test[['SibSp', 'Parch','Fare']])))

0.6910112359550562


# mixed naive bayes

#### 3. Từ bộ dữ liệu liên minh hãy xây dựng thuật toán Naive Bayes bằng cách dùng tất cả features (mix cả Multinomial và Gaussian. Lưu ý: Khỏi chia tập train, test. Dự đoán thẳng trên tập train) (2đ)

In [35]:
#Chuẩn bị data cho câu 3
import pandas as pd
df_lienminh = pd.read_csv("https://raw.githubusercontent.com/dinhvietcuong1996/Lab-MachineLearningCourse/master/Lab04/lienminh.csv")
df_lienminh.head()

,killsDiff,minionsKilledDiff,wardPlacedDiff,firstBlood,heralds,dragons,teamWins
0,3,-2,13,blue,none,none,red
1,0,-66,0,red,red,red,red
2,-4,-17,0,red,none,blue,red
3,-1,-34,28,red,blue,none,red
4,0,-15,58,red,none,red,red


Về cơ bản, Liên Minh Huyền Thoại là một game giữa hai đội Blue và Red với mục tiêu là phá hủy nhà chính đối phương. Mỗi đội có 5 người chơi, mỗi người điều khiển 1 nhân vật. Trong trò chơi, để chiến thắng người chơi có thể:

Hạ gục đối phương: thể hiện bằng chỉ số hạ gục (Kills)

Hạ gục lính kiếm tiền mua trang bị (MinionsKilled)

Cắm mắt để có tầm nhìn, thêm thông tin về đối phương (WardsPlaced)

Ăn sứ giả lấy bùa và vật phẩm (Heralds)

Ăn bùa rồng (Dragons)

Mạng đầu tiên (firstBlood)

Đội thắng (teamWins)

In [36]:
df_lienminh.nunique()

killsDiff             33
minionsKilledDiff    213
wardPlacedDiff       284
firstBlood             2
heralds                3
dragons                3
teamWins               2
dtype: int64

In [37]:
df_lienminh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9879 entries, 0 to 9878
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   killsDiff          9879 non-null   int64 
 1   minionsKilledDiff  9879 non-null   int64 
 2   wardPlacedDiff     9879 non-null   int64 
 3   firstBlood         9879 non-null   object
 4   heralds            9879 non-null   object
 5   dragons            9879 non-null   object
 6   teamWins           9879 non-null   object
dtypes: int64(3), object(4)
memory usage: 540.4+ KB


In [38]:
#Phân tích thử các giá trị có trong mỗi feature
for i in df_lienminh.columns:
    print(i)
    print(df_lienminh[i].unique())

killsDiff
[  3   0  -4  -1   2   1  -8   6  -5   8   5  -9  -2  -3  -6  -7   4   9
   7 -11  11 -13  10 -10  12  13 -12  16 -15  14 -14 -17  15]
minionsKilledDiff
[  -2  -66  -17  -34  -15    4   61   52  -51  -27   15  -31  -19  -18
  -28   53  -11   -6    7   18   11  -24   26    8    1   12   -3  -21
   17  -23    9   46   10  -60   55  -38  -35    3   31    5   38   19
   23  -37  -25   54   -4  -52   -1  -43   14   24   37   -5   32   16
   44   21   28   41  -47  -22  112  -46   64   39  -13   27   13   20
   -8  -70  -16  -42  -10  -36  -12   85  -26   36   -7  -50   35  -54
   29  -41   74  -14  -20  -32  -63  -49    6  -71   33  -29    2   47
   40   34  -53  -68  -45   50   63   22   43  -40   25 -101    0  -75
  -39  104   96  -62   56   30   62   82   45  -61   59   69   42  -58
  -93   76   65   57   49  -30  -44   51   79  -59   -9  110  -72   48
  -76   72  -64  -33 -120   60  -84   77  -73  -48  -83  -98  -92   84
   71  -81  -57  -74   80  -82   73  -65  100   89   83 

In [39]:
X = df_lienminh.drop(['teamWins'],axis = 1)
y = df_lienminh['teamWins']

In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X,y ,test_size=0.2
                                                    , random_state=42)

# y_train = y_train.values
y_test = y_test.values

In [41]:
print(X_train.shape)
print(X_test.shape)

(7903, 6)
(1976, 6)


In [42]:
def Mix_model(X,y):
    X_cate = X.select_dtypes(include=['object'])
    X_num = X.select_dtypes(exclude=['object'])
    # Calculate Prior Probability
    y_unique = y.unique()
    prior_probability = {}
    for i in range(len(y_unique)):
        prior_probability[y_unique[i]] = sum(y == y_unique[i])/len(y)
    # Calculate Conditional Probability
    _, prob_cat = CategoricalNB_model(X_cate,y)
    _, prob_num = GaussianNB_model(X_num,y)
    return prior_probability, prob_cat, prob_num

def prediction_Mix(X_test, prob_cat, prob_num, prior_probability):
    X_test_cate = X_test.select_dtypes(include=['object'])
    X_test_num = X_test.select_dtypes(exclude=['object'])
    y_pred = []
    for i in range(len(X_test)):
        p0 = list(prior_probability.values())[0] # P(teamWins= red)
        p1 = list(prior_probability.values())[1] # P(teamWins= blue)
        for col in X_test_num.columns:
            p0 = p0*gaussian(X_test_num[col].values[i], prob_num[col]['Mean'][0],prob_num[col]['Std'][0])
            p1 = p1*gaussian(X_test_num[col].values[i], prob_num[col]['Mean'][1],prob_num[col]['Std'][1])
        for col in X_test_cate.columns:        
            p0 = p0*prob_cat[col][X_test_cate[col].values[i]][0]
            p1 = p1*prob_cat[col][X_test_cate[col].values[i]][1]
        if p0 > p1:
            y_pred.append('red')
        else:
            y_pred.append('blue')
    return np.array(y_pred)
prior_probability, prob_cat, prob_num = Mix_model(X, y)

In [43]:
print(prior_probability)

{'red': 0.5009616357930965, 'blue': 0.4990383642069035}


In [44]:
y_pred = prediction_Mix(X, prob_cat, prob_num, prior_probability)
y_pred

array(['blue', 'red', 'red', ..., 'red', 'red', 'blue'], dtype='<U4')

In [45]:
np.mean(y_pred == y)

0.7116104868913857

In [46]:
data_train = Encoder(X_train)
data_test = Encoder(X_test)

<ipython-input-22-e4fe50279b9b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column] = labelencoder.transform(data[column])


In [47]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
LE.fit(y_train)
label_train = LE.transform(y_train)
label_test = LE.transform(y_test)

In [48]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7903 entries, 6392 to 7270
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   killsDiff          7903 non-null   int64
 1   minionsKilledDiff  7903 non-null   int64
 2   wardPlacedDiff     7903 non-null   int64
 3   firstBlood         7903 non-null   int32
 4   heralds            7903 non-null   int32
 5   dragons            7903 non-null   int32
dtypes: int32(3), int64(3)
memory usage: 339.6 KB


In [49]:
from sklearn.naive_bayes import GaussianNB
GNB = GaussianNB().fit(data_train, label_train)
GNB.score(data_test,label_test)

0.722165991902834

In [50]:
from mixed_naive_bayes import MixedNB
clf = MixedNB().fit(data_train,label_train)
print(clf.predict(data_test)[:100])
print(label_test[:100])

[]
[1 0 0 1 1 1 0 0 0 0 0 1 1 1 0 0 0 0 1 1 1 1 0 0 1 0 1 1 1 0 1 1 0 0 1 1 0
 1 1 0 1 0 1 0 0 0 1 0 1 1 1 0 1 0 1 0 1 0 0 1 0 0 0 1 0 0 0 1 1 1 0 0 1 1
 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 0 1 0 1 0 0 0 1 0 1 0]
[1 0 0 0 1 1 0 1 0 0 0 0 1 1 0 0 0 0 1 0 0 1 1 1 1 0 0 1 1 0 0 1 0 0 0 1 0
 0 0 0 1 0 1 0 0 0 1 1 1 1 1 0 1 0 1 0 1 0 0 1 1 0 0 0 1 0 0 1 0 1 1 1 1 0
 0 1 1 0 1 1 0 0 0 0 1 1 0 1 1 0 1 1 1 0 0 0 0 1 1 0]


In [51]:
clf.score(data_test,label_test)

0.722165991902834